In [1]:
%matplotlib widget
import torch
import pickle
import os
from sklearn.cluster import DBSCAN
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tqdm.notebook import tqdm
import network.cpc
from network.cpc import CDCK2
from torch.utils import data
from utils.MatplotlibUtils import reduce_dims_and_plot
from soft_decision_tree.sdt_model import SDT


# Load the model and the dataset

In [2]:
model_path = r'C:\Users\eitan\OneDrive - Technion\Desktop\tsne\negative from sample\with_data\knn_loss_batch_512_k_32\models\epoch_40.pt'
dataset_path = r'C:\Users\eitan\OneDrive - Technion\Desktop\tsne\negative from sample\with_data\knn_loss_batch_512_k_32\data\test_data.file'
batch_size = 32
print(f"Load the model from: {model_path}")
model = torch.load(model_path, map_location='cpu')

with open(dataset_path, 'rb') as fp:
    dataset = pickle.load(fp)
    
loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

Load the model from: C:\Users\eitan\OneDrive - Technion\Desktop\tsne\negative from sample\with_data\knn_loss_batch_512_k_32\models\epoch_40.pt


# Extract representations

In [3]:
projections = torch.tensor([])
model = model.eval()
device = 'cpu'
with torch.no_grad():
    bar = tqdm(total=len(loader.dataset))
    for batch in loader:
        hidden = CDCK2.init_hidden(len(batch))
        batch = batch.to(device)
        hidden = hidden.to(device)

        y = model.predict(batch, hidden).detach().cpu()
        projections = torch.cat([projections, y])
        bar.update(y.shape[0])

# Cluster the data

In [9]:
clustering  = DBSCAN(eps=1.2).fit(projections)
labels = set(clustering.labels_)
print(labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1}


# Visualize with T-SNE

In [ ]:
for perplexity in [50, 100, 200]:
    reduce_dims_and_plot(projections,
                         y=clustering.labels_,
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='3d',
                         show_figure=True,
                         close_figure=False,
                         text=None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

In [ ]:
lr = 1e-3
weight_decay = 5e-4

In [ ]:
tree = SDT(input_dim=projections.shape[1], output_dim=len(labels), depth=12, lamda=1e-3, use_cuda=False)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
tree = tree.to(device)

In [ ]:
loader = data.DataLoader(projections, batch_size=batch_size, shuffle=False)

In [ ]:
for epoch in range(epochs):
    print(f"Classes: {tree.get_classes()}")
    # Training
    tree.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        batch_size = data.size()[0]
        data, target = data.to(device), target.to(device)
        target_onehot = onehot_coding(target, device, output_dim)

        output, penalty = tree.forward(data)

        # Loss
        loss = criterion(output, target.view(-1))

        # Penalty
        loss += penalty

        # L1
        fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
        l1_regularization = torch.norm(fc_params, 1)
        loss += args.sparsity_lamda * l1_regularization

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training status
        if batch_idx % log_interval == 0:
            pred = output.data.max(1)[1]
            correct = pred.eq(target.view(-1).data).sum()

            msg = (
                "Epoch: {:02d} | Batch: {:03d} / {:03d} | Loss: {:.5f} |"
                " Accuracy: {:03f}"
            )
            print(msg.format(epoch, batch_idx, len(train_loader), loss.item(), correct.item() / batch_size))
            training_loss_list.append(loss.cpu().data.numpy())